# Eksplorasi Data PODES 2024 - Blok V, VI, VII, X
**Analisis Fasilitas Lingkungan, Bencana, Transportasi, dan TIK untuk Wilayah 3579**

Notebook ini menganalisis data PODES 2024 untuk:
- **Blok V**: Fasilitas Lingkungan & Perumahan (air, sanitasi, energi, sampah)
- **Blok VI**: Bencana Alam & Mitigasi 
- **Blok VII**: Akses & Transportasi
- **Blok X**: Teknologi Informasi & Komunikasi

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")

## 1. Load & Preprocessing Data

In [ ]:
# Load main PODES data
def load_podes_file(filename):
    """Load PODES CSV with proper encoding and data types"""
    try:
        df = pd.read_csv(f'../data/raw/{filename}', dtype=str, encoding='utf-8-sig')
        print(f"✅ {filename}: {df.shape[0]} rows, {df.shape[1]} columns")
        return df
    except FileNotFoundError:
        print(f"❌ {filename} not found")
        return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error loading {filename}: {e}")
        return pd.DataFrame()

# Load all PODES files
podes_files = ['podes01_3579.csv', 'podes02_3579.csv', 'podes03_3579.csv', 'podes04_3579.csv']
podes_dfs = []

for file in podes_files:
    df = load_podes_file(file)
    if not df.empty:
        podes_dfs.append(df)

print(f"\n📊 Loaded {len(podes_dfs)} PODES files successfully")

In [ ]:
# Merge all PODES data
if podes_dfs:
    # Start with first dataframe
    df_merged = podes_dfs[0].copy()
    
    # Merge with other dataframes
    for i, df in enumerate(podes_dfs[1:], 1):
        if 'IDDESA' in df.columns:
            # Merge on IDDESA
            before_merge = df_merged.shape
            df_merged = df_merged.merge(df, on='IDDESA', how='outer', suffixes=('', f'_dup{i}'))
            after_merge = df_merged.shape
            
            # Remove duplicate columns
            dup_cols = [col for col in df_merged.columns if f'_dup{i}' in col]
            df_merged = df_merged.drop(columns=dup_cols)
            
            print(f"Merged with PODES{i+1}: {before_merge} → {after_merge}")
    
    print(f"\n📊 Final merged dataset: {df_merged.shape[0]} desa, {df_merged.shape[1]} variables")
else:
    print("❌ No PODES files loaded")
    df_merged = pd.DataFrame()

In [ ]:
# Data quality check
if not df_merged.empty:
    print("📋 Data Quality Summary:")
    print(f"Total rows: {len(df_merged):,}")
    print(f"Total columns: {len(df_merged.columns):,}")
    
    # Check key identity columns
    identity_cols = ['IDDESA', 'NAMA_KEC', 'NAMA_DESA']
    for col in identity_cols:
        if col in df_merged.columns:
            missing = df_merged[col].isnull().sum()
            unique = df_merged[col].nunique()
            print(f"{col}: {missing} missing, {unique} unique values")
    
    # Overall completeness
    total_cells = len(df_merged) * len(df_merged.columns)
    missing_cells = df_merged.isnull().sum().sum()
    completeness = ((total_cells - missing_cells) / total_cells) * 100
    print(f"\nData completeness: {completeness:.1f}%")
    
    # Show sample
    print("\n📝 Sample data (first 3 rows):")
    display(df_merged[['IDDESA', 'NAMA_KEC', 'NAMA_DESA']].head(3))

## 3. Load Indicators Catalog & Apply Mapping

In [ ]:
# Load indicators catalog
try:
    catalog = pd.read_csv('../indicators_catalog.csv', encoding='utf-8-sig')
    print(f"✅ Indicators catalog loaded: {len(catalog)} indicators")
    
    # Parse JSON mappings
    def safe_json_parse(json_str):
        try:
            return json.loads(json_str) if pd.notna(json_str) else {}
        except:
            return {}
    
    catalog['values_map_parsed'] = catalog['values_map'].apply(safe_json_parse)
    
    # Show catalog summary
    print("\n📊 Catalog by theme:")
    theme_counts = catalog['theme'].value_counts()
    for theme, count in theme_counts.items():
        print(f"{theme}: {count} indicators")
    
    display(catalog[['theme', 'indicator_code', 'indicator_name', 'aggregation']].head())
    
except Exception as e:
    print(f"❌ Error loading catalog: {e}")
    catalog = pd.DataFrame()

In [ ]:
# Apply value mappings to key variables
def apply_mapping(series, mapping_dict):
    """Apply mapping with fallback to original values"""
    if not mapping_dict:
        return series
    
    mapped = series.map(mapping_dict)
    return mapped.fillna(series)

# Create labeled version of data
df_labeled = df_merged.copy()

# Apply mappings for visualization
key_variables = ['R508A', 'R506', 'R504A', 'R504D', 'R504E', 'R505', 'R1006D', 'R1001B1']

mapped_count = 0
for var in key_variables:
    if var in df_labeled.columns and not catalog.empty:
        # Get mapping for this variable
        var_row = catalog[catalog['variable'] == var]
        if not var_row.empty:
            mapping = var_row.iloc[0]['values_map_parsed']
            if mapping:
                df_labeled[f'{var}_label'] = apply_mapping(df_labeled[var], mapping)
                mapped_count += 1
                print(f"✅ Applied mapping to {var}")

print(f"\n📊 Successfully mapped {mapped_count} variables")

## 4. Exploratory Data Analysis - Core Indicators

In [ ]:
# Analyze core indicators
print("🔍 CORE INDICATORS ANALYSIS")
print("=" * 50)

core_indicators = {
    'R508A': 'Sumber Air Minum',
    'R506': 'Jenis Jamban', 
    'R504D': 'TPS3R',
    'R504E': 'Bank Sampah',
    'R505': 'Pemilahan Sampah'
}

for var_code, var_name in core_indicators.items():
    if var_code in df_merged.columns:
        print(f"\n📊 {var_name} ({var_code}):")
        
        # Basic frequency
        freq = df_merged[var_code].value_counts().head()
        total = df_merged[var_code].notna().sum()
        
        for value, count in freq.items():
            pct = (count / total) * 100
            print(f"  {value}: {count} desa ({pct:.1f}%)")
        
        # Missing data
        missing = df_merged[var_code].isnull().sum()
        if missing > 0:
            print(f"  Missing: {missing} desa ({(missing/len(df_merged)*100):.1f}%)")
    else:
        print(f"\n❌ {var_name} ({var_code}): Variable not found")

## 5. Data Visualization

In [ ]:
# Visualization 1: Sumber Air Minum Distribution
if 'R508A' in df_labeled.columns:
    print("📊 VISUALISASI 1: Distribusi Sumber Air Minum")
    
    # Use labeled version if available
    if 'R508A_label' in df_labeled.columns:
        air_data = df_labeled['R508A_label'].value_counts()
    else:
        air_data = df_labeled['R508A'].value_counts()
    
    fig_air = px.bar(
        x=air_data.index,
        y=air_data.values,
        title="Distribusi Sumber Air Minum Utama",
        labels={'x': 'Sumber Air', 'y': 'Jumlah Desa'},
        color=air_data.values,
        color_continuous_scale='Blues'
    )
    fig_air.update_layout(xaxis_tickangle=-45, showlegend=False)
    fig_air.show()
    
    # Summary
    total_desa = air_data.sum()
    print(f"Total desa dengan data: {total_desa}")
    print(f"Sumber terbanyak: {air_data.index[0]} ({air_data.iloc[0]} desa, {(air_data.iloc[0]/total_desa*100):.1f}%)")

In [ ]:
# Visualization 2: Persampahan (TPS3R, Bank Sampah, Pemilahan)
print("📊 VISUALISASI 2: Program Pengelolaan Sampah")

sampah_programs = ['R504D', 'R504E', 'R505']
sampah_labels = ['TPS3R', 'Bank Sampah', 'Pemilahan']
sampah_data = []

for var, label in zip(sampah_programs, sampah_labels):
    if var in df_merged.columns:
        # Count "Ada" (kode 1)
        ada_count = (df_merged[var] == '1').sum()
        total_count = df_merged[var].notna().sum()
        percentage = (ada_count / total_count * 100) if total_count > 0 else 0
        
        sampah_data.append({
            'Program': label,
            'Jumlah_Desa': ada_count,
            'Persentase': percentage
        })

if sampah_data:
    sampah_df = pd.DataFrame(sampah_data)
    
    fig_sampah = px.bar(
        sampah_df,
        x='Program',
        y='Persentase',
        title="Persentase Desa dengan Program Pengelolaan Sampah",
        labels={'Persentase': 'Persentase Desa (%)'},
        color='Persentase',
        color_continuous_scale='Greens',
        text='Persentase'
    )
    fig_sampah.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
    fig_sampah.update_layout(showlegend=False)
    fig_sampah.show()
    
    print("\nRingkasan Program Sampah:")
    for _, row in sampah_df.iterrows():
        print(f"{row['Program']}: {row['Jumlah_Desa']} desa ({row['Persentase']:.1f}%)")

In [ ]:
# Visualization 3: Internet & Jalan Infrastructure
print("📊 VISUALISASI 3: Infrastruktur Internet & Jalan")

# Internet Quality
if 'R1006D' in df_labeled.columns:
    if 'R1006D_label' in df_labeled.columns:
        internet_data = df_labeled['R1006D_label'].value_counts()
    else:
        internet_data = df_labeled['R1006D'].value_counts()
    
    fig_internet = px.pie(
        values=internet_data.values,
        names=internet_data.index,
        title="Distribusi Kualitas Internet Seluler"
    )
    fig_internet.show()

# Road Surface
if 'R1001B1' in df_labeled.columns:
    if 'R1001B1_label' in df_labeled.columns:
        jalan_data = df_labeled['R1001B1_label'].value_counts()
    else:
        jalan_data = df_labeled['R1001B1'].value_counts()
    
    fig_jalan = px.bar(
        x=jalan_data.index,
        y=jalan_data.values,
        title="Distribusi Jenis Permukaan Jalan Utama",
        labels={'x': 'Jenis Permukaan', 'y': 'Jumlah Desa'},
        color=jalan_data.values,
        color_continuous_scale='Oranges'
    )
    fig_jalan.update_layout(xaxis_tickangle=-45, showlegend=False)
    fig_jalan.show()

## 6. Key Performance Indicators (KPI)

In [ ]:
# Calculate KPIs
print("🎯 KEY PERFORMANCE INDICATORS")
print("=" * 50)

def calculate_kpi_rate(df, variable, true_values={'1'}):
    """Calculate percentage of desa with positive indicator"""
    if variable not in df.columns:
        return 0.0
    
    total = df[variable].notna().sum()
    if total == 0:
        return 0.0
    
    positive = df[variable].isin(true_values).sum()
    return (positive / total) * 100

# Define KPIs
kpis = {
    'TPS3R': calculate_kpi_rate(df_merged, 'R504D'),
    'Bank Sampah': calculate_kpi_rate(df_merged, 'R504E'),
    'Pemilahan Sampah': calculate_kpi_rate(df_merged, 'R505'),
    'Internet Kantor Desa': calculate_kpi_rate(df_merged, 'R1005B'),
    'BTS Tersedia': calculate_kpi_rate(df_merged, 'R1006A'),
    'Angkutan Umum': calculate_kpi_rate(df_merged, 'R1001C')
}

# Proxy KPIs
proxy_kpis = {}

# Air Layak (bukan sungai/danau)
if 'R508A' in df_merged.columns:
    air_layak = df_merged['R508A'] != '6'  # 6 = Sungai/Danau
    total_air = df_merged['R508A'].notna().sum()
    proxy_kpis['Air Layak (proxy)'] = (air_layak.sum() / total_air * 100) if total_air > 0 else 0

# Sanitasi Layak (ada jamban)
if 'R506' in df_merged.columns:
    sanitasi_layak = df_merged['R506'] != '4'  # 4 = Tidak Ada Jamban
    total_sanitasi = df_merged['R506'].notna().sum()
    proxy_kpis['Sanitasi Layak (proxy)'] = (sanitasi_layak.sum() / total_sanitasi * 100) if total_sanitasi > 0 else 0

# Internet 4G+
if 'R1006D' in df_merged.columns:
    internet_4g = df_merged['R1006D'].isin(['1', '2'])  # 1=4G+, 2=4G
    total_internet = df_merged['R1006D'].notna().sum()
    proxy_kpis['Internet 4G+ (proxy)'] = (internet_4g.sum() / total_internet * 100) if total_internet > 0 else 0

# Jalan Aspal/Beton
if 'R1001B1' in df_merged.columns:
    jalan_aspal = df_merged['R1001B1'] == '1'  # 1 = Aspal/Beton
    total_jalan = df_merged['R1001B1'].notna().sum()
    proxy_kpis['Jalan Aspal (proxy)'] = (jalan_aspal.sum() / total_jalan * 100) if total_jalan > 0 else 0

# Combine all KPIs
all_kpis = {**kpis, **proxy_kpis}

# Display KPIs
print("\n📊 HASIL KPI:")
for kpi_name, kpi_value in all_kpis.items():
    print(f"{kpi_name}: {kpi_value:.1f}%")

# Summary statistics
kpi_values = list(all_kpis.values())
print(f"\n📈 STATISTIK KPI:")
print(f"Rata-rata: {np.mean(kpi_values):.1f}%")
print(f"Median: {np.median(kpi_values):.1f}%")
print(f"Min: {np.min(kpi_values):.1f}%")
print(f"Max: {np.max(kpi_values):.1f}%")

## 7. Analysis by Kecamatan

In [ ]:
# Analysis by Kecamatan
if 'NAMA_KEC' in df_merged.columns:
    print("🗺️ ANALISIS PER KECAMATAN")
    print("=" * 50)
    
    kecamatan_list = df_merged['NAMA_KEC'].dropna().unique()
    print(f"Total kecamatan: {len(kecamatan_list)}")
    
    # Kecamatan summary
    kec_summary = []
    for kec in kecamatan_list:
        kec_data = df_merged[df_merged['NAMA_KEC'] == kec]
        
        # Calculate key KPIs for this kecamatan
        kec_kpis = {
            'Kecamatan': kec,
            'Jumlah_Desa': len(kec_data),
            'TPS3R_Pct': calculate_kpi_rate(kec_data, 'R504D'),
            'Bank_Sampah_Pct': calculate_kpi_rate(kec_data, 'R504E'),
            'Pemilahan_Pct': calculate_kpi_rate(kec_data, 'R505'),
            'Internet_4G_Pct': (kec_data['R1006D'].isin(['1', '2']).sum() / kec_data['R1006D'].notna().sum() * 100) if 'R1006D' in kec_data.columns and kec_data['R1006D'].notna().sum() > 0 else 0,
            'Jalan_Aspal_Pct': calculate_kpi_rate(kec_data, 'R1001B1')
        }
        kec_summary.append(kec_kpis)
    
    kec_df = pd.DataFrame(kec_summary)
    kec_df = kec_df.sort_values('Jumlah_Desa', ascending=False)
    
    print("\n📊 Ringkasan per Kecamatan:")
    display(kec_df.round(1))
    
    # Best and worst performing kecamatan
    if len(kec_df) > 1:
        # Calculate average score
        score_cols = [col for col in kec_df.columns if col.endswith('_Pct')]
        kec_df['Avg_Score'] = kec_df[score_cols].mean(axis=1)
        
        best_kec = kec_df.loc[kec_df['Avg_Score'].idxmax()]
        worst_kec = kec_df.loc[kec_df['Avg_Score'].idxmin()]
        
        print(f"\n🥇 Kecamatan Terbaik: {best_kec['Kecamatan']} (Skor: {best_kec['Avg_Score']:.1f}%)")
        print(f"🔻 Kecamatan Terlemah: {worst_kec['Kecamatan']} (Skor: {worst_kec['Avg_Score']:.1f}%)")
else:
    print("❌ Data kecamatan tidak tersedia")
    kec_df = pd.DataFrame()

## 8. Data Export for Streamlit Application

In [ ]:
# Create processed data folder
import os
os.makedirs('../data/processed', exist_ok=True)

print("📤 EKSPOR DATA UNTUK APLIKASI STREAMLIT")
print("=" * 50)

In [ ]:
# Export 1: Per-desa view with core indicators
if not df_merged.empty:
    # Select core columns for per-desa view
    core_cols = ['IDDESA', 'NAMA_KEC', 'NAMA_DESA']
    
    # Add core indicators
    indicator_cols = ['R508A', 'R508B', 'R512', 'R506', 'R507', 'R514',
                     'R504A', 'R504D', 'R504E', 'R505',
                     'R501', 'R502', 'R503A',
                     'R1005B', 'R1006A', 'R1006C', 'R1006D',
                     'R1001A', 'R1001B1', 'R1001B2', 'R1001C',
                     'R513', 'R515', 'R516', 'R601', 'R604']
    
    # Filter existing columns
    available_cols = core_cols + [col for col in indicator_cols if col in df_merged.columns]
    
    per_desa_df = df_merged[available_cols].copy()
    
    # Add calculated proxy indicators
    if 'R508A' in per_desa_df.columns:
        per_desa_df['air_layak'] = (per_desa_df['R508A'] != '6').astype(int)
    
    if 'R506' in per_desa_df.columns:
        per_desa_df['sanitasi_layak'] = (per_desa_df['R506'] != '4').astype(int)
    
    if 'R1006D' in per_desa_df.columns:
        per_desa_df['internet_4g_plus'] = per_desa_df['R1006D'].isin(['1', '2']).astype(int)
    
    if 'R1001B1' in per_desa_df.columns:
        per_desa_df['jalan_aspal'] = (per_desa_df['R1001B1'] == '1').astype(int)
    
    # Export
    per_desa_df.to_csv('../data/processed/per_desa_view.csv', index=False, encoding='utf-8-sig')
    print(f"✅ per_desa_view.csv exported: {len(per_desa_df)} rows, {len(per_desa_df.columns)} columns")
else:
    print("❌ Cannot export per_desa_view: no data available")

In [ ]:
# Export 2: Aggregated kecamatan data
if not kec_df.empty:
    agg_kec_df = kec_df.copy()
    agg_kec_df.to_csv('../data/processed/agg_kecamatan.csv', index=False, encoding='utf-8-sig')
    print(f"✅ agg_kecamatan.csv exported: {len(agg_kec_df)} rows, {len(agg_kec_df.columns)} columns")
else:
    print("❌ Cannot export agg_kecamatan: no kecamatan data available")

In [ ]:
# Export 3: KPI summary
kpi_summary_df = pd.DataFrame([
    {'indicator': k, 'value': v, 'type': 'direct' if k in kpis else 'proxy'}
    for k, v in all_kpis.items()
])

kpi_summary_df.to_csv('../data/processed/kpi_summary.csv', index=False, encoding='utf-8-sig')
print(f"✅ kpi_summary.csv exported: {len(kpi_summary_df)} KPIs")

## 9. Executive Summary & Insights

In [ ]:
print("📋 EXECUTIVE SUMMARY")
print("=" * 50)

if not df_merged.empty:
    print(f"\n🎯 DATASET OVERVIEW:")
    print(f"• Total desa: {len(df_merged):,}")
    if 'NAMA_KEC' in df_merged.columns:
        print(f"• Total kecamatan: {df_merged['NAMA_KEC'].nunique()}")
    print(f"• Total variabel: {len(df_merged.columns)}")
    print(f"• Kelengkapan data: {completeness:.1f}%")
    
    print(f"\n🏆 TOP PERFORMERS:")
    top_3_kpis = sorted(all_kpis.items(), key=lambda x: x[1], reverse=True)[:3]
    for i, (kpi, value) in enumerate(top_3_kpis, 1):
        print(f"{i}. {kpi}: {value:.1f}%")
    
    print(f"\n⚠️ AREAS FOR IMPROVEMENT:")
    bottom_3_kpis = sorted(all_kpis.items(), key=lambda x: x[1])[:3]
    for i, (kpi, value) in enumerate(bottom_3_kpis, 1):
        print(f"{i}. {kpi}: {value:.1f}%")
    
    print(f"\n💡 KEY INSIGHTS:")
    
    # Insight 1: Persampahan
    if all(k in all_kpis for k in ['TPS3R', 'Bank Sampah', 'Pemilahan Sampah']):
        avg_sampah = np.mean([all_kpis['TPS3R'], all_kpis['Bank Sampah'], all_kpis['Pemilahan Sampah']])
        if avg_sampah < 30:
            print(f"• Pengelolaan sampah masih rendah (rata-rata {avg_sampah:.1f}%) - perlu program intensif")
        else:
            print(f"• Pengelolaan sampah cukup baik (rata-rata {avg_sampah:.1f}%) - pertahankan dan tingkatkan")
    
    # Insight 2: Digital divide
    if 'Internet 4G+ (proxy)' in all_kpis:
        internet_4g = all_kpis['Internet 4G+ (proxy)']
        if internet_4g < 50:
            print(f"• Kesenjangan digital masih tinggi - hanya {internet_4g:.1f}% desa dengan 4G+")
        else:
            print(f"• Konektivitas internet cukup baik - {internet_4g:.1f}% desa dengan 4G+")
    
    # Insight 3: Infrastructure
    if 'Jalan Aspal (proxy)' in all_kpis:
        jalan_aspal = all_kpis['Jalan Aspal (proxy)']
        if jalan_aspal < 60:
            print(f"• Infrastruktur jalan perlu perhatian - hanya {jalan_aspal:.1f}% desa dengan jalan aspal")
        else:
            print(f"• Infrastruktur jalan relatif baik - {jalan_aspal:.1f}% desa dengan jalan aspal")
    
    print(f"\n✅ Data telah diekspor dan siap untuk aplikasi Streamlit")
    print(f"📁 File lokasi: data/processed/")
    
else:
    print("❌ Tidak dapat membuat summary - data tidak tersedia")

---
**📊 End of EDA - PODES 2024 Wilayah 3579**

Data telah dianalisis dan diekspor untuk digunakan dalam aplikasi Streamlit. File output tersedia di folder `data/processed/`:

- `per_desa_view.csv`: Data detail per desa dengan indikator inti
- `agg_kecamatan.csv`: Data agregasi per kecamatan
- `kpi_summary.csv`: Ringkasan semua KPI

**Next Steps:**
1. Jalankan aplikasi Streamlit: `streamlit run app.py`
2. Eksplorasi dashboard interaktif
3. Analisis lebih mendalam per desa/kecamatan